In [1]:
!pip install -r requirements.txt

  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6086 sha256=ea08b2f99db4271314a0ea18c5a51f8859f2bc0326360fbb89de8caefca067a9
  Stored in directory: c:\users\hapsi\appdata\local\pip\cache\wheels\64\80\6e\caa3e16deb0267c3cbfd36862058a724144e19fdb9eb03af0f
  Created wheel for nvidia-ml-py3: filename=nvidia_ml_py3-7.352.0-py3-none-any.whl size=19196 sha256=00418d5da52445f1f7cd5038ec3bfa6894c7397d4396fe3a2b86b8a4bfc81c9c
  Stored in directory: c:\users\hapsi\appdata\local\pip\cache\wheels\df\99\da\c34f202dc8fd1dffd35e0ecf1a7d7f8374ca05fbcbaf974b83
  Created wheel for gql: filename=gql-0.2.0-py3-none-any.whl size=7633 sha256=dcf965252688cda089203b00d4047a9fb970115b222f83fed9cfd555f03ab758
  Stored in directory: c:\users\hapsi\appdata\local\pip\cache\wheels\b6\9a\56\5456fd32264a8fc53eefcb2f74e24e99a7ef4eb40a9af5c905
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6492 sha256=9dd1efd1ca6c8ebf2b3bf9688747e9f964c03da17bf8541fb0693d82a2fb9

ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Отказано в доступе: 'C:\\ProgramData\\Anaconda3\\Lib\\site-packages\\__pycache__\\nvidia_smi.cpython-37.pyc'
Consider using the `--user` option or check the permissions.



In [7]:
import fastai
from deoldify.visualize import *

torch.backends.cudnn.benchmark = True

In [9]:
from deoldify.generators import gen_inference_deep, gen_inference_wide

root_folder:Path=Path('./')
weights_name:str='ColorizeStable_gen'

learn = gen_inference_wide(root_folder=root_folder, weights_name=weights_name)

Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /Users/hapsidra/.cache/torch/checkpoints/resnet101-5d3b4d8f.pth


In [10]:
model = learn.model
model.cpu()
input_names = ["my_input"]
output_names = ["my_output"]

In [12]:
def remove_all_spectral_norm(item):
    if isinstance(item, nn.Module):
        try:
            nn.utils.remove_spectral_norm(item)
        except Exception:
            pass
        
        for child in item.children():  
            remove_all_spectral_norm(child)

    if isinstance(item, nn.ModuleList):
        for module in item:
            remove_all_spectral_norm(module)

    if isinstance(item, nn.Sequential):
        modules = item.children()
        for module in modules:
            remove_all_spectral_norm(module)
remove_all_spectral_norm(model)

In [13]:

dummy_input = torch.randn(1, 3, 224, 224)

torch.onnx.export(model, dummy_input, "colorizer.onnx",
                  verbose=True)

graph(%input.1 : Float(1, 3, 224, 224),
      %layers.0.0.weight : Float(64, 3, 7, 7),
      %layers.0.1.weight : Float(64),
      %layers.0.1.bias : Float(64),
      %layers.0.1.running_mean : Float(64),
      %layers.0.1.running_var : Float(64),
      %layers.0.1.num_batches_tracked : Long(),
      %layers.0.4.0.conv1.weight : Float(64, 64, 1, 1),
      %layers.0.4.0.bn1.weight : Float(64),
      %layers.0.4.0.bn1.bias : Float(64),
      %layers.0.4.0.bn1.running_mean : Float(64),
      %layers.0.4.0.bn1.running_var : Float(64),
      %layers.0.4.0.bn1.num_batches_tracked : Long(),
      %layers.0.4.0.conv2.weight : Float(64, 64, 3, 3),
      %layers.0.4.0.bn2.weight : Float(64),
      %layers.0.4.0.bn2.bias : Float(64),
      %layers.0.4.0.bn2.running_mean : Float(64),
      %layers.0.4.0.bn2.running_var : Float(64),
      %layers.0.4.0.bn2.num_batches_tracked : Long(),
      %layers.0.4.0.conv3.weight : Float(256, 64, 1, 1),
      %layers.0.4.0.bn3.weight : Float(256),
      %layer